# How to share state between threads

By default, state is scoped to a single thread. LangGraph also lets you customize the scope for a given key-value pair. You can use this to share information between threads.

For instance, you can persist each user’s preferences to shared state and re-use them in new conversational threads.

In this notebook, we will show how to construct and use such a graph.

## Setup

First, let's install the required packages and set our API keys

In [4]:
%%capture --no-stderr
%pip install -U langchain_openai langgraph

In [1]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")

<div class="admonition tip">
    <p class="admonition-title">Set up <a href="https://smith.langchain.com">LangSmith</a> for LangGraph development</p>
    <p style="padding-top: 5px;">
        Sign up for LangSmith to quickly spot issues and improve the performance of your LangGraph projects. LangSmith lets you use trace data to debug, test, and monitor your LLM apps built with LangGraph — read more about how to get started <a href="https://docs.smith.langchain.com">here</a>. 
    </p>
</div>    

## Create graph

In this example we will create a graph that will let us store information about a user's preferences. We will do so by defining a state key that will be scoped to a user_id, and allowing the model to populate this field as it deems fit (by providing the model with a tool to save information about the user).

    
<div class="admonition note">
    <p class="admonition-title">Typing shared state keys</p>
    <p style="margin-top: 5px;">
        Shared state channels (keys) MUST be dictionaries (see <code>info</code> channel in the State example below)
    </p>
</div>

In [5]:
from langgraph.graph.message import MessagesState
from langgraph.graph.state import StateGraph
from langgraph.store.memory import MemoryStore
from langgraph.managed.shared_value import SharedValue
from typing import Literal, TypedDict, Annotated
import uuid
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver


class State(MessagesState):
    # We use an info key to track information
    # This is scoped to a user_id, so it will be information specific to each user
    info: Annotated[dict, SharedValue.on("user_id")]


# We will give this as a tool to the agent
# This will let the agent call this tool to save a fact
class Info(TypedDict):
    """This tool should be called when you want to save a new fact about the user.

    Attributes:
        fact (str): A fact about the user.
        topic (str): The topic related the fact is about, i.e. Food, Location, Movies, etc.
    """

    fact: str
    topic: str


# This is the prompt we give the agent
# We will pass known info into the prompt
# We will tell it to use the Info tool to save more
prompt = """You are a helpful assistant that learns about users to provide better assistance.

Current user information:
<info>
{info}
</info>

Instructions:
1. Use the `Info` tool to save new information the user shares.
2. Save facts, opinions, preferences, and experiences.
3. Your goal: Improve assistance by building a user profile over time.

Remember: Every piece of information helps you serve the user better in future interactions.
"""


# We give the model access to the Info tool
model = ChatOpenAI().bind_tools([Info])


def call_model(state: State):
    """Call the model."""
    # The info value here is scoped to the user_id
    info = "\n".join([d["fact"] for d in state["info"].values()])
    # Format system prompt
    system_msg = prompt.format(info=info)
    # Call model
    response = model.invoke(
        [{"role": "system", "content": system_msg}] + state["messages"]
    )
    return {"messages": [response]}


# Routing function to decide what to do next
# If no tool calls, then we end
# If tool calls, then we update memory
def route(state) -> Literal["__end__", "update_memory"]:
    if len(state["messages"][-1].tool_calls) == 0:
        return "__end__"
    else:
        return "update_memory"


def update_memory(state: State):
    """Update the memory."""
    tool_calls = []
    memories = {}
    # Each tool call is a new memory to save
    for tc in state["messages"][-1].tool_calls:
        # We append ToolMessages (to pass back to the LLM)
        # This is needed because OpenAI requires each tool call be followed by a ToolMessage
        tool_calls.append(
            {"role": "tool", "content": "Saved!", "tool_call_id": tc["id"]}
        )
        # We create a new memory from this tool call
        memories[str(uuid.uuid4())] = {
            "fact": tc["args"]["fact"],
            "topic": tc["args"]["topic"],
        }
    # Return the messages and memories to update the state with
    return {"messages": tool_calls, "info": memories}


# This is the in memory checkpointer we will use
# We need this because we want to enable threads (conversations)
memory = MemorySaver()

# This is the in memory Key Value store
# This is needed to save the memories
kv = MemoryStore()

# Construct this relatively simple graph
graph = StateGraph(State)
graph.add_node(call_model)
graph.add_node(update_memory)
graph.add_edge("update_memory", "__end__")
graph.add_edge("__start__", "call_model")
graph.add_conditional_edges("call_model", route, ["__end__", "update_memory"])
graph = graph.compile(checkpointer=memory, store=kv)

## Run graph on one thread

We can now run the graph on one thread and give it some information

In [6]:
config = {"configurable": {"thread_id": "1", "user_id": "1"}}

# First let's just say hi to the AI
for update in graph.stream(
    {"messages": [{"role": "user", "content": "hi"}]}, config, stream_mode="updates"
):
    print(update)

# Let's continue the conversation (by passing the same config) and tell the AI we like pepperoni pizza
for update in graph.stream(
    {"messages": [{"role": "user", "content": "i like pepperoni pizza"}]},
    config,
    stream_mode="updates",
):
    print(update)

# Let's continue the conversation even further (by passing the same config) and tell the AI we live in SF
for update in graph.stream(
    {"messages": [{"role": "user", "content": "i also just moved to SF"}]},
    config,
    stream_mode="updates",
):
    print(update)

{'call_model': {'messages': [AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 181, 'total_tokens': 191, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-865472b7-68e0-4b93-bf63-13bd1dc4f3f0-0', usage_metadata={'input_tokens': 181, 'output_tokens': 10, 'total_tokens': 191})]}}
{'call_model': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_BcSTNM6xueW6lgcaA8GdBuy4', 'function': {'arguments': '{"fact":"likes pepperoni pizza","topic":"Food"}', 'name': 'Info'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 203, 'total_tokens': 223, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None,

## Run graph on a different thread

We can now run the graph on a different thread and see that it remembers facts about the user (specifically that the user likes pepperoni pizza and lives in SF):

In [7]:
config = {"configurable": {"thread_id": "2", "user_id": "1"}}

for update in graph.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": "where and what should i eat for dinner? Can you list some restaurants?",
            }
        ]
    },
    config,
    stream_mode="updates",
):
    print(update)

{'call_model': {'messages': [AIMessage(content="I can help with that! Since you just moved to San Francisco, how about trying some local favorites? Here are a few restaurants you might enjoy:\n\n1. Tony's Pizza Napoletana - Known for their delicious pepperoni pizza.\n2. The House - Offers a mix of Asian fusion dishes.\n3. Tadich Grill - A historic seafood restaurant with a cozy atmosphere.\n4. Zuni Cafe - Famous for its roast chicken and innovative cuisine.\n5. La Taqueria - A popular spot for authentic Mexican tacos.\n\nFeel free to explore these options and let me know if you'd like more recommendations or information about any specific cuisine!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 131, 'prompt_tokens': 206, 'total_tokens': 337, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8530de74-bc07-4b31-b58d-4f4c06

Perfect! The AI recommended restaurants in SF, and included a pizza restaurant at the top of it's list.

Notice that the `messages` in this new thread do NOT contain the messages from the previous thread since we didn't store them as shared values across the `user_id`. However, the `info` we saved in the previous thread was saved since we passed in the same `user_id` in this new thread.

Let's now run the graph for another user to verify that the preferences of the first user are self contained:

In [8]:
config = {"configurable": {"thread_id": "3", "user_id": "2"}}

for update in graph.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": "where and what should i eat for dinner? Can you list some restaurants?",
            }
        ]
    },
    config,
    stream_mode="updates",
):
    print(update)

{'call_model': {'messages': [AIMessage(content='I can help you with that! Could you please provide me with your location or a preferred cuisine for dinner?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 195, 'total_tokens': 218, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3f7eaa92-d3f0-4cca-ab13-0ecd7b922b9a-0', usage_metadata={'input_tokens': 195, 'output_tokens': 23, 'total_tokens': 218})]}}


Perfect! The graph has forgotten all of the previous preferences and has to ask the user for it's location and dietary preferences.